# Dataset description
Without Ni, Ti, Tl

In [1]:
from ase.db import connect
import pandas as pd
from ase.io import read
import seaborn as sns
import numpy as np
import prince
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ase.visualize import view

In [2]:
slabs = ['Cu','Ag','Au','Pt','Sn','In','Pb','Cd','Fe','Zn', 'Ni']
# removed Ni, Tl, Ti, Ir, Rh, Ru, Hg, Pd

MMA_OCCOH_db = connect('/raw/MMA_adsOCCOH_missing_features.db')
MMA_C_db = connect('/raw/MMA_adsC_missing_features.db')
MMA_O_db = connect('/raw/MMA_adsO_missing_features.db')
MMA_OH_db = connect('/raw/MMA_adsOH_missing_features.db')
MMA_clean_db = connect('/raw/MMA_clean_isolated.db')

MMA_OCCOH_ads = [row.toatoms() for row in MMA_OCCOH_db.select()]
MMA_C_ads = [row.toatoms() for row in MMA_C_db.select()]
MMA_O_ads = [row.toatoms() for row in MMA_O_db.select()]
MMA_OH_ads = [row.toatoms() for row in MMA_OH_db.select()]
MMA_clean = [row.toatoms() for row in MMA_clean_db.select()]

MMA_OCCOH_list = [row.symbols[2] for row in MMA_OCCOH_ads]
MMA_C_list = [row.symbols[2] for row in MMA_C_ads]
MMA_O_list = [row.symbols[2] for row in MMA_O_ads]
MMA_OH_list = [row.symbols[2] for row in MMA_OH_ads]
MMA_clean_list = [row.symbols[2] for row in MMA_clean]

MMA_isolated_molecules_db = connect('/raw/MMA_isolated_molecules_BEEF.db')
MMA_isolated_molecules = [row.toatoms() for row in MMA_isolated_molecules_db.select()]
MMA_CO2 = MMA_isolated_molecules[0].get_potential_energy()
MMA_H2 = MMA_isolated_molecules[1].get_potential_energy()
MMA_H2O = MMA_isolated_molecules[2].get_potential_energy()
MMA_CO = MMA_isolated_molecules[3].get_potential_energy()


BAG_isolated_molecules_db = connect('/raw/BAG_Molecules.db')
BAG_CO= BAG_isolated_molecules_db.get(Molecule='CO').energy
BAG_CO2= BAG_isolated_molecules_db.get(Molecule='CO2').energy
BAG_H2= BAG_isolated_molecules_db.get(Molecule='H2').energy
BAG_CH2O= BAG_isolated_molecules_db.get(Molecule='CH2O').energy


OperationalError: unable to open database file

In [41]:
for i in MMA_OCCOH_ads:
    view(i)

python(47178) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47189) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47200) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47201) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47202) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47203) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47204) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47210) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47211) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47212) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(47214) Malloc

In [10]:
print(len(MMA_OCCOH_list))

17


In [33]:
print(f'MMA CO: {MMA_CO}')
print(f'BAG CO: {BAG_CO}')

print(f'MMA CO2: {MMA_CO2}')
print(f'BAG CO2: {BAG_CO2}')

print(f'MMA H2: {MMA_H2}')
print(f'BAG H2: {BAG_H2}')


MMA CO: -34.804522403953655
BAG CO: -34.78708730478091
MMA CO2: -54.7702399031112
BAG CO2: -54.74388643001491
MMA H2: -8.006927454257369
BAG H2: -7.988302799256201


In [40]:
clean_slab_rows = []

for slab in slabs:
    MMA_OCCOH_index = next((i for i, s in enumerate(MMA_OCCOH_list) if s.startswith(slab)), None)
    MMA_C_index = next((i for i, s in enumerate(MMA_C_list) if s.startswith(slab)), None)
    MMA_O_index = next((i for i, s in enumerate(MMA_O_list) if s.startswith(slab)), None)
    MMA_OH_index = next((i for i, s in enumerate(MMA_OH_list) if s.startswith(slab)), None)
    MMA_clean_index = next((i for i, s in enumerate(MMA_clean_list) if s.startswith(slab)), None)

    BAG_clean_potential = read(f'../raw_data/BAG_Metals_large.db@Metal={slab},ads=Clean')[0].get_potential_energy()
    MMA_clean_potential = MMA_clean[MMA_clean_index].get_potential_energy()

    print(f'{slab}')
    print(f'BAG: {BAG_clean_potential}')
    print(f'MMA: {MMA_clean_potential}')
    print(np.abs(((BAG_clean_potential-MMA_clean_potential)/MMA_clean_potential)*100))
    #print(read(f'../raw_data/BAG_Metals_large.db@Metal={slab},ads=H')[0].get_potential_energy() - BAG_clean_potential - BAG_H2 / 2)
    
    clean_slab_rows.append({
        "metal": slab,
        "facet": "(p111)",
        "H*": read(f'../raw_data/BAG_Metals_large.db@Metal={slab},ads=H')[0].get_potential_energy() - BAG_clean_potential - BAG_H2 / 2,
        "CO*": read(f'../raw_data/BAG_Metals_large.db@Metal={slab},ads=CO')[0].get_potential_energy() - BAG_clean_potential - BAG_CO,
#        "COOH*": read(f'../raw_data/BAG_Metals_large.db@Metal={slab},ads=HCOO')[0].get_potential_energy() - BAG_clean_potential - BAG_H2 / 2 - BAG_CO2,
        "OCCOH*": MMA_OCCOH_ads[MMA_OCCOH_index].get_potential_energy() - MMA_clean_potential - 2 * MMA_CO - MMA_H2 / 2,
        "OH*": MMA_OH_ads[MMA_OH_index].get_potential_energy() - MMA_clean_potential - MMA_H2O + MMA_H2 / 2,
        "O*": MMA_O_ads[MMA_O_index].get_potential_energy() - MMA_clean_potential - MMA_H2O + MMA_H2,
        "C*": MMA_C_ads[MMA_C_index].get_potential_energy() - MMA_clean_potential - MMA_CO + MMA_H2O - MMA_H2,
        })

Cu
BAG: -2725.9524397200303
MMA: -2727.537042455015
0.05809646983047308
Ag
BAG: -5335.838731817584
MMA: -5338.249716100774
0.0451643218547564
Au
BAG: -12027.66484518725
MMA: -12029.063782679994
0.011629645648387105
Pt
BAG: -11877.077732407612
MMA: -11878.930540670248
0.015597433256240786
Sn
BAG: -5885.171222413575
MMA: -5886.384889080151
0.020618200974711447
In
BAG: -5674.20034363921
MMA: -5675.755802352887
0.0274053142496505
Pb
BAG: -12820.73649308934
MMA: -12821.935064199635
0.009347817660079924
Cd
BAG: -5438.288217707233
MMA: -5439.733614979937
0.026571103936467933
Fe
BAG: -2514.3088220354202
MMA: -2515.360698828161
0.041818129432913034
Zn
BAG: -2776.9986107273003
MMA: -2777.790008141639
0.02849018147589807
Ni
BAG: -2715.157761532084
MMA: -2716.3103265399905
0.0424312714436677


TypeError: list indices must be integers or slices, not NoneType

In [4]:
clean_df = pd.DataFrame(clean_slab_rows)

metal_facet =  [str(clean_df.iloc[id, 0]) + str(clean_df.iloc[id, 1]) for id in range(len(clean_df))]
clean_df.insert(0, "metal_facet", metal_facet)

In [5]:
CH4_dict = {'Cu(p111)': 33.3, 'Ag(p111)': 0, 'Au(p111)': 0, 'Pt(p111)': 0, 'Sn(p111)': 0, 'In(p111)': 0, 'Pb(p111)': 0, 'Cd(p111)': 1.3, 'Fe(p111)': 0, 'Zn(p111)': 0}
clean_df["CH4"] = clean_df.apply(lambda row: CH4_dict[row.metal_facet], axis=1)

C2H4_dict = {'Cu(p111)': 25.5, 'Ag(p111)': 0, 'Au(p111)': 0, 'Pt(p111)': 0, 'Sn(p111)': 0, 'In(p111)': 0, 'Pb(p111)': 0, 'Cd(p111)': 0, 'Fe(p111)': 0, 'Zn(p111)': 0}
clean_df["C2H4"] = clean_df.apply(lambda row: C2H4_dict[row.metal_facet], axis=1)

EtOH_dict = {'Cu(p111)': 5.7, 'Ag(p111)': 0, 'Au(p111)': 0, 'Pt(p111)': 0, 'Sn(p111)': 0, 'In(p111)': 0, 'Pb(p111)': 0, 'Cd(p111)': 0, 'Fe(p111)': 0, 'Zn(p111)': 0}
clean_df["EtOH"] = clean_df.apply(lambda row: EtOH_dict[row.metal_facet], axis=1)

PrOH_dict = {'Cu(p111)': 3, 'Ag(p111)': 0, 'Au(p111)': 0, 'Pt(p111)': 0, 'Sn(p111)': 0, 'In(p111)': 0, 'Pb(p111)': 0, 'Cd(p111)': 0, 'Fe(p111)': 0, 'Zn(p111)': 0}
clean_df["PrOH"] = clean_df.apply(lambda row: PrOH_dict[row.metal_facet], axis=1)

CO_dict = {'Cu(p111)': 1.3, 'Ag(p111)': 81.5, 'Au(p111)': 87.1, 'Pt(p111)': 0, 'Sn(p111)': 7.1, 'In(p111)': 2.1, 'Pb(p111)': 0, 'Cd(p111)': 13.9, 'Fe(p111)': 0, 'Zn(p111)': 79.4}
clean_df["CO"] = clean_df.apply(lambda row: CO_dict[row.metal_facet], axis=1)

HCOO_dict = {'Cu(p111)': 9.4, 'Ag(p111)': 0.8, 'Au(p111)': 0.7, 'Pt(p111)': 0.1, 'Sn(p111)': 88.4, 'In(p111)': 94.9, 'Pb(p111)': 97.4, 'Cd(p111)': 78.4, 'Fe(p111)': 0, 'Zn(p111)': 6.1}
clean_df["HCOOH"] = clean_df.apply(lambda row: HCOO_dict[row.metal_facet], axis=1)

H2_dict = {'Cu(p111)': 20.5, 'Ag(p111)': 12.4, 'Au(p111)': 10.2, 'Pt(p111)': 95.7, 'Sn(p111)': 4.6, 'In(p111)': 3.3, 'Pb(p111)': 5, 'Cd(p111)': 9.4, 'Fe(p111)': 94.8, 'Zn(p111)': 9.9}
clean_df["H2"] = clean_df.apply(lambda row: H2_dict[row.metal_facet], axis=1)



#for ../data_ex_Ni_Tl_Ti_Ir_Rh_Ru_COOH_OCCOH
# clean_df['CH4'] = [33.3,0,0,0,2.9,0,0,0,1.3,0,0,0]
# clean_df['C2H4'] = [25.5,0,0,0,0,0,0,0,0,0,0,0]
# clean_df['EtOH'] = [5.7,0,0,0,0,0,0,0,0,0,0,0]
# clean_df['PrOH'] = [3,0,0,0,0,0,0,0,0,0,0,0]
# clean_df['CO'] = [1.3,81.5,87.1,0,28.3,7.1,2.1,0,13.9,0,0,79.4]
# clean_df['HCOOH'] = [9.4,0.8,0.7,0.1,2.8,88.4,94.9,97.4,78.4,0,99.5,6.1]
# clean_df['H2'] = [20.5,12.4,10.2,95.7,26.2,4.6,3.3,5,9.4,94.8,0,9.9]

#clean_df['HCOOH'] = [cu,ag,au,pt,gd,sn,in,pb,cd,fe,hg,zn]

In [7]:
clean_df

,metal_facet,metal,facet,H*,CO*,OCCOH*,OH*,O*,C*,CH4,C2H4,EtOH,PrOH,CO,HCOO,H2
0,Cu(111),Cu,(111),0.023236,-0.556853,0.070148,0.856654,0.936462,1.807598,33.3,25.5,5.7,3,1.3,9.4,20.5
1,Ag(111),Ag,(111),0.433729,-0.087450,1.025489,1.280763,2.010691,3.185689,0.0,0.0,0.0,0,81.5,0.8,12.4
2,Au(111),Au,(111),0.407140,-0.090951,0.605229,1.977928,2.656480,2.423984,0.0,0.0,0.0,0,87.1,0.7,10.2
3,Pt(111),Pt,(111),-0.225918,-1.430356,-1.688655,0.774783,1.584325,-0.444057,0.0,0.0,0.0,0,0.0,0.1,95.7
4,Sn(111),Sn,(111),0.826178,-0.165051,0.381047,0.352010,0.551014,1.354276,0.0,0.0,0.0,0,7.1,88.4,4.6
5,In(111),In,(111),0.848249,-0.141406,0.659065,0.171168,0.363054,1.846257,0.0,0.0,0.0,0,2.1,94.9,3.3
6,Pb(111),Pb,(111),0.907909,-0.123199,0.973498,0.386810,1.028115,2.258035,0.0,0.0,0.0,0,0.0,97.4,5.0
7,Cd(111),Cd,(111),0.731254,-0.124407,0.805581,0.949977,0.612106,1.898795,1.3,0.0,0.0,0,13.9,78.4,9.4
8,Fe(111),Fe,(111),-0.459489,-1.669840,-1.884714,-0.229531,-0.862073,-0.629994,0.0,0.0,0.0,0,0.0,0.0,94.8
9,Zn(111),Zn,(111),0.533896,-0.148887,0.250439,0.702620,-0.153392,1.033256,0.0,0.0,0.0,0,79.4,6.1,9.9


In [6]:
facet_df = pd.read_csv('../raw_data/BAG_cu_facets.csv')
facet_df['facet'] = facet_df['facet'].astype(str).apply(lambda x: '(' + x + ')')
facet_df.drop(["COOH*"], axis=1)
metal_facet =  [str(facet_df.iloc[id, 0]) + str(facet_df.iloc[id, 1]) for id in range(len(facet_df))]
facet_df.insert(0, "metal_facet", metal_facet)

In [11]:
facet_df

,metal_facet,metal,facet,OH*,OCCOH*,O*,H*,COOH*,CO*,C*,...,CO,H2,MeD,EtOH,PrD,AlOH,PrOH,HCOOH,CH3OOH,Unnamed: 20
0,Cu(100),Cu,(100),-0.075690,-0.165608,0.597152,0.138277,-0.145655,-0.569521,0.652417,...,0.9,6.8,1.6,9.7,2.8,0.8,1.50,3.0,1.0,98.90
1,Cu(1111),Cu,(1111),-0.113445,-0.524108,0.493543,0.068053,-0.424328,-0.754738,0.734585,...,1.8,8.8,1.1,14.4,3.3,1.0,2.30,3.2,2.1,97.10
2,Cu(911),Cu,(911),-0.323448,-0.501938,0.439509,0.060410,-0.412155,-0.737806,0.722656,...,0.0,12.7,1.2,16.9,2.4,2.5,4.50,3.5,1.1,101.40
3,Cu(711),Cu,(711),-0.305812,-0.479831,0.528111,0.031482,-0.411630,-0.716354,0.756368,...,1.1,15.6,1.2,7.4,5.2,2.2,4.60,4.6,0.9,97.80
4,Cu(511),Cu,(511),-0.106930,-0.407334,0.552297,0.065944,-0.397774,-0.699135,0.803959,...,1.9,18.1,1.4,12.2,3.0,1.6,3.30,8.8,0.8,101.50
5,Cu(311),Cu,(311),-0.340910,-0.369563,0.671110,0.022122,-0.469174,-0.775473,0.835478,...,2.6,13.3,1.1,3.3,2.3,0.4,1.50,14.0,0.6,98.90
6,Cu(111),Cu,(111),0.067849,0.100497,0.885095,0.019614,-0.189064,-0.559284,1.731727,...,6.4,16.3,2.1,2.6,0.6,0.7,0.00,11.5,1.5,96.30
7,Cu(810),Cu,(810),-0.042859,-0.083884,0.671217,0.124735,-0.141256,-0.681575,0.610553,...,1.4,8.7,0.9,26.0,1.1,0.9,1.90,1.5,1.6,95.50
8,Cu(610),Cu,(610),-0.139494,-0.077392,0.501558,0.051596,-0.158980,-0.686766,0.680591,...,0.9,9.0,1.5,26.0,1.2,1.3,2.00,1.4,1.6,97.20
9,Cu(510),Cu,(510),-0.159645,-0.207617,0.500368,0.087414,-0.174253,-0.713403,0.597684,...,2.1,10.5,3.0,26.1,2.6,1.7,1.70,2.8,2.1,103.00


In [9]:
df = pd.concat([clean_df.iloc[:,:], facet_df[['metal_facet', 'metal', 'facet', 'OH*', 'O*', 'OCCOH*', 'H*', 'CO*', 'C*', 'CH4', 'C2H4', 'CO', 'H2', 'EtOH', 'PrOH', 'HCOOH']]], ignore_index=True)

Vnhe_dict = {'Cu(p111)': -1.44, 'Ag(p111)': -1.37, 'Au(p111)': -1.14, 'Pt(p111)': -1.07, 'Sn(p111)': -1.48, 'In(p111)': -1.55, 'Pb(p111)': -1.63, 'Cd(p111)': -1.63, 'Fe(p111)': -0.91, 'Zn(p111)': -1.54, 'Cu(100)': -1.4, 'Cu(1111)': -1.37, 'Cu(911)': -1.36, 'Cu(711)': -1.34, 'Cu(511)': -1.36, 'Cu(311)': -1.37, 'Cu(111)': -1.55, 'Cu(810)': -1.38, 'Cu(610)': -1.37, 'Cu(510)': -1.38, 'Cu(310)': -1.42, 'Cu(210)': -1.52, 'Cu(755)': -1.43, 'Cu(533)': -1.42, 'Cu(211)': -1.38, 'Cu(332)': -1.51, 'Cu(331)': -1.55, 'Cu(110)': -1.58, 'Cu(650)': -1.59, 'Cu(320)': -1.52, 'Cu(210)': -1.52}
df["Vnhe"] = df.apply(lambda row: Vnhe_dict[row.metal_facet], axis=1)

vnhe_column = df["Vnhe"]
df.drop(columns=["Vnhe"], inplace=True)
desired_position = 3
df.insert(desired_position, "Vnhe", vnhe_column)

# df.insert(loc=2, column='V_she', value=[-1.44,-1.37,-1.14,-1.07,-1.2,-1.48,-1.55,-1.63,-1.63,-0.91,-1.51,-1.54,-1.4,-1.37,-1.36,-1.34,-1.36,-1.37,-1.55,-1.38,-1.37,-1.38,-1.42,-1.52,-1.43,-1.42,1.38,-1.51,-1.55,-1.58,-1.59,-1.52,-1.52])

In [10]:
df

,metal_facet,metal,facet,Vnhe,H*,CO*,OCCOH*,OH*,O*,C*,CH4,C2H4,EtOH,PrOH,CO,HCOOH,H2
0,Cu(p111),Cu,(p111),-1.44,0.023236,-0.556853,0.070148,0.856654,0.936462,1.807598,33.3,25.5,5.7,3.00,1.3,9.4,20.5
1,Ag(p111),Ag,(p111),-1.37,0.433729,-0.087450,1.025489,1.280763,2.010691,3.185689,0.0,0.0,0.0,0.00,81.5,0.8,12.4
2,Au(p111),Au,(p111),-1.14,0.407140,-0.090951,0.605229,1.977928,2.656480,2.423984,0.0,0.0,0.0,0.00,87.1,0.7,10.2
3,Pt(p111),Pt,(p111),-1.07,-0.225918,-1.430356,-1.688655,0.774783,1.584325,-0.444057,0.0,0.0,0.0,0.00,0.0,0.1,95.7
4,Sn(p111),Sn,(p111),-1.48,0.826178,-0.165051,0.381047,0.352010,0.551014,1.354276,0.0,0.0,0.0,0.00,7.1,88.4,4.6
5,In(p111),In,(p111),-1.55,0.848249,-0.141406,0.659065,0.171168,0.363054,1.846257,0.0,0.0,0.0,0.00,2.1,94.9,3.3
6,Pb(p111),Pb,(p111),-1.63,0.907909,-0.123199,0.973498,0.386810,1.028115,2.258035,0.0,0.0,0.0,0.00,0.0,97.4,5.0
7,Cd(p111),Cd,(p111),-1.63,0.731254,-0.124407,0.805581,0.949977,0.612106,1.898795,1.3,0.0,0.0,0.00,13.9,78.4,9.4
8,Fe(p111),Fe,(p111),-0.91,-0.459489,-1.669840,-1.884714,-0.229531,-0.862073,-0.629994,0.0,0.0,0.0,0.00,0.0,0.0,94.8
9,Zn(p111),Zn,(p111),-1.54,0.533896,-0.148887,0.250439,0.702620,-0.153392,1.033256,0.0,0.0,0.0,0.00,79.4,6.1,9.9


In [11]:
df.to_csv("../data_ex_Ni_Tl_Ti_Ir_Rh_Ru_Hg_Pd_COOH-Incl_Vnhe")